In [45]:
#pip install ta

In [50]:
import pandas as pd
from datetime import datetime

# Define the path to the file
file_path = './yahoo_appl/preprocessed_data/processed_files/aapl_historical_data_Max_Historical_Prices_Daily_preprocessed.csv'

# Read the data from CSV file
data = pd.read_csv(file_path)


In [51]:
data

,Date,Open,High,Low,Close*,Adj Close**,Volume,MA_5,MA_10,MA_20,Pct_change,Volatility,Day_of_week
0,2023-11-03,174.24,176.82,173.35,176.65,176.65,79763700,NaN,NaN,NaN,NaN,NaN,Friday
1,2023-11-02,175.52,177.78,175.46,177.57,177.57,77334800,NaN,NaN,NaN,0.520804,NaN,Thursday
2,2023-11-01,171.00,174.23,170.12,173.97,173.97,56934900,NaN,NaN,NaN,-2.027369,NaN,Wednesday
3,2023-10-31,169.35,170.90,167.90,170.77,170.77,44846000,NaN,NaN,NaN,-1.839398,NaN,Tuesday
4,2023-10-30,169.02,171.17,168.87,170.29,170.29,51131000,173.850,NaN,NaN,-0.281080,NaN,Monday
...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,2022-11-11,145.82,150.01,144.37,149.70,149.06,93979700,149.506,149.619,147.0510,0.957648,1.068562,Friday
246,2022-11-10,141.24,146.87,139.50,146.87,146.25,118854000,148.736,149.495,147.2865,-1.890448,1.314888,Thursday
247,2022-11-09,138.50,138.55,134.59,134.87,134.30,74917800,145.952,147.875,146.8975,-8.170491,3.725627,Wednesday
248,2022-11-08,140.41,141.43,137.49,139.50,138.91,89908500,143.844,146.807,146.8255,3.432935,4.331353,Tuesday


In [91]:
import pandas as pd
import os
import ta

def preprocess_data(file_path):
    # Read the data from the CSV file
    data = pd.read_csv(file_path)

    # Convert 'Date' to datetime format
    data['Date'] = pd.to_datetime(data['Date'], errors='coerce')

    # Convert columns to numeric, coercing errors into NaN (which will then be filled)
    numeric_cols = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
    for col in numeric_cols:
        if data[col].dtype == 'object':
            data[col] = pd.to_numeric(data[col].str.replace(',', ''), errors='coerce')

    # Handle missing values by forward fill
    data.fillna(method='ffill', inplace=True)

    # Now convert 'Volume' to int, since we have no NaN values anymore
    data['Volume'] = data['Volume'].astype(int)

    # Calculate moving averages for 'Close' column
    data['MA_5'] = data['Close'].rolling(window=5).mean()
    data['MA_10'] = data['Close'].rolling(window=10).mean()
    data['MA_12'] = data['Close'].rolling(window=12).mean()
    data['MA_26'] = data['Close'].rolling(window=26).mean()    
    data['MA_20'] = data['Close'].rolling(window=20).mean()
    data['MA_50'] = data['Close'].rolling(window=50).mean()
    data['MA_100'] = data['Close'].rolling(window=100).mean()

    # Calculate percentage change for 'Close' column
    data['Pct_change'] = data['Close'].pct_change() * 100

    # Calculate volatility (standard deviation of percentage change for 'Close')
    data['Volatility'] = data['Pct_change'].rolling(window=5).std()

    # Create a column for the day of the week
    data['Day_of_week'] = data['Date'].dt.day_name()

    # Calculate RSI
    data['RSI'] = ta.momentum.RSIIndicator(data['Close'], window=14).rsi()

    # Calculate MACD
    macd = ta.trend.MACD(data['Close'])
    data['MACD'] = macd.macd()
    data['MACD_signal'] = macd.macd_signal()

    # Calculate Bollinger Bands
    bollinger = ta.volatility.BollingerBands(data['Close'], window=20, window_dev=2)
    data['Bollinger_mavg'] = bollinger.bollinger_mavg()
    data['Bollinger_hband'] = bollinger.bollinger_hband()
    data['Bollinger_lband'] = bollinger.bollinger_lband()

    # Adding Lagged Features
    data['Lag_1'] = data['Close'].shift(1)
    data['Lag_2'] = data['Close'].shift(2)
    data['Lag_3'] = data['Close'].shift(3)

    # Add volume as a percentage of moving average volume
    data['Volume_MA_10'] = data['Volume'].rolling(window=10).mean()
    data['Volume_pct_MA'] = data['Volume'] / data['Volume_MA_10']

    # Calculate the inter-day price range
    data['High_Low_Range'] = data['High'] - data['Low']

    # Price change direction (1 for up, 0 for down)
    data['Price_Change_Direction'] = (data['Close'] > data['Close'].shift(1)).astype(int)
    
    
        
    # Calculate the MACD Histogram as a new feature
    data['MACD_Hist'] = data['MACD'] - data['MACD_signal']

    # Calculate the 12-period and 26-period moving averages of the closing price

    # Calculate the Percentage Price Oscillator (PPO)
    data['PPO'] = ((data['MA_12'] - data['MA_26']) / data['MA_26']) * 100

    # Add a dummy example for Fundamental Ratio (P/E Ratio) - you would replace this with actual data if you have it
    data['P/E_Ratio'] = np.random.uniform(low=10, high=30, size=data.shape[0])  # This is just a placeholder

    # Normalize Volume (you can decide on the normalization technique that is best for your analysis)
    data['Volume_Norm'] = (data['Volume'] - data['Volume'].mean()) / data['Volume'].std()

    # Add VIX as an example for an external market indicator - you would replace this with actual data if you have it
    data['VIX'] = np.random.uniform(low=10, high=40, size=data.shape[0])  # This is just a placeholder

    # Fill any remaining NaN values after shifting
    data.fillna(method='ffill', inplace=True)
    data.fillna(method='bfill', inplace=True)

    # Drop the initial rows that have NaN values due to moving averages and other indicators
    data.dropna(inplace=True)

    # Define a new directory for preprocessed files
    preprocessed_dir = './yahoo_/yahoo_preprocessed/'
    os.makedirs(preprocessed_dir, exist_ok=True)

    # Save the preprocessed data back to CSV in a single preprocessed directory
    preprocessed_file_name = os.path.basename(file_path).replace('.csv', '_preprocessed.csv')
    preprocessed_file_path = os.path.join(preprocessed_dir, preprocessed_file_name)
    data.to_csv(preprocessed_file_path, index=False)

    return preprocessed_file_path




In [92]:
file_paths = [
'yahoo_/yahoo_sp500/CSCO.csv',
'yahoo_/yahoo_sp500/UAL.csv',
'yahoo_/yahoo_sp500/TROW.csv',
'yahoo_/yahoo_sp500/ISRG.csv',
'yahoo_/yahoo_sp500/NVR.csv',
'yahoo_/yahoo_sp500/TPR.csv',
'yahoo_/yahoo_sp500/DVN.csv',
'yahoo_/yahoo_sp500/CE.csv',
'yahoo_/yahoo_sp500/MRO.csv',
'yahoo_/yahoo_sp500/BA.csv',
'yahoo_/yahoo_sp500/VRTX.csv',
'yahoo_/yahoo_sp500/GILD.csv',
'yahoo_/yahoo_sp500/EQIX.csv',
'yahoo_/yahoo_sp500/TER.csv',
'yahoo_/yahoo_sp500/PODD.csv',
'yahoo_/yahoo_sp500/MDT.csv',
'yahoo_/yahoo_sp500/V.csv',
'yahoo_/yahoo_sp500/QRVO.csv',
'yahoo_/yahoo_sp500/A.csv',
'yahoo_/yahoo_sp500/FOX.csv',
'yahoo_/yahoo_sp500/FLT.csv',
'yahoo_/yahoo_sp500/MO.csv',
'yahoo_/yahoo_sp500/CTRA.csv',
'yahoo_/yahoo_sp500/SWKS.csv',
'yahoo_/yahoo_sp500/ENPH.csv',
'yahoo_/yahoo_sp500/MCHP.csv',
'yahoo_/yahoo_sp500/CDNS.csv',
'yahoo_/yahoo_sp500/MSCI.csv',
'yahoo_/yahoo_sp500/CHTR.csv',
'yahoo_/yahoo_sp500/EIX.csv',
'yahoo_/yahoo_sp500/KDP.csv',
'yahoo_/yahoo_sp500/BBY.csv',
'yahoo_/yahoo_sp500/GEN.csv',
'yahoo_/yahoo_sp500/WBA.csv',
'yahoo_/yahoo_sp500/LVS.csv',
'yahoo_/yahoo_sp500/HCA.csv',
'yahoo_/yahoo_sp500/AJG.csv',
'yahoo_/yahoo_sp500/DTE.csv',
'yahoo_/yahoo_sp500/C.csv',
'yahoo_/yahoo_sp500/T.csv',
'yahoo_/yahoo_sp500/CF.csv',
'yahoo_/yahoo_sp500/MGM.csv',
'yahoo_/yahoo_sp500/HUM.csv',
'yahoo_/yahoo_sp500/CBOE.csv',
'yahoo_/yahoo_sp500/CFG.csv',
'yahoo_/yahoo_sp500/APH.csv',
'yahoo_/yahoo_sp500/SYY.csv',
'yahoo_/yahoo_sp500/MSI.csv',
'yahoo_/yahoo_sp500/FCX.csv',
'yahoo_/yahoo_sp500/ADM.csv',
'yahoo_/yahoo_sp500/LH.csv',
'yahoo_/yahoo_sp500/LNT.csv',
'yahoo_/yahoo_sp500/BAC.csv',
'yahoo_/yahoo_sp500/PSX.csv',
'yahoo_/yahoo_sp500/GPN.csv',
'yahoo_/yahoo_sp500/HUBB.csv',
'yahoo_/yahoo_sp500/PPG.csv',
'yahoo_/yahoo_sp500/TECH.csv',
'yahoo_/yahoo_sp500/IRM.csv',
'yahoo_/yahoo_sp500/IQV.csv',
'yahoo_/yahoo_sp500/ESS.csv',
'yahoo_/yahoo_sp500/WBD.csv',
'yahoo_/yahoo_sp500/HAL.csv',
'yahoo_/yahoo_sp500/STZ.csv',
'yahoo_/yahoo_sp500/BG.csv',
'yahoo_/yahoo_sp500/PARA.csv',
'yahoo_/yahoo_sp500/ADI.csv',
'yahoo_/yahoo_sp500/F.csv',
'yahoo_/yahoo_sp500/ADBE.csv',
'yahoo_/yahoo_sp500/STLD.csv',
'yahoo_/yahoo_sp500/CPRT.csv',
'yahoo_/yahoo_sp500/TDG.csv',
'yahoo_/yahoo_sp500/TFX.csv',
'yahoo_/yahoo_sp500/ULTA.csv',
'yahoo_/yahoo_sp500/ARE.csv',
'yahoo_/yahoo_sp500/SYK.csv',
'yahoo_/yahoo_sp500/KVUE.csv',
'yahoo_/yahoo_sp500/CB.csv',
'yahoo_/yahoo_sp500/TSN.csv',
'yahoo_/yahoo_sp500/GNRC.csv',
'yahoo_/yahoo_sp500/PEP.csv',
'yahoo_/yahoo_sp500/PEG.csv',
'yahoo_/yahoo_sp500/NOW.csv',
'yahoo_/yahoo_sp500/LLY.csv',
'yahoo_/yahoo_sp500/COST.csv',
'yahoo_/yahoo_sp500/REG.csv',
'yahoo_/yahoo_sp500/NWS.csv',
'yahoo_/yahoo_sp500/LOW.csv',
'yahoo_/yahoo_sp500/MDLZ.csv',
'yahoo_/yahoo_sp500/BKNG.csv',
'yahoo_/yahoo_sp500/ZBRA.csv',
'yahoo_/yahoo_sp500/FMC.csv',
'yahoo_/yahoo_sp500/XEL.csv',
'yahoo_/yahoo_sp500/AIZ.csv',
'yahoo_/yahoo_sp500/MET.csv',
'yahoo_/yahoo_sp500/FTV.csv',
'yahoo_/yahoo_sp500/DLR.csv',
'yahoo_/yahoo_sp500/ACGL.csv',
'yahoo_/yahoo_sp500/XRAY.csv',
'yahoo_/yahoo_sp500/FAST.csv',
'yahoo_/yahoo_sp500/TJX.csv',
'yahoo_/yahoo_sp500/SNA.csv',
'yahoo_/yahoo_sp500/MPC.csv',
'yahoo_/yahoo_sp500/BR.csv',
'yahoo_/yahoo_sp500/D.csv',
'yahoo_/yahoo_sp500/MRK.csv',
'yahoo_/yahoo_sp500/STX.csv',
'yahoo_/yahoo_sp500/NOC.csv',
'yahoo_/yahoo_sp500/BXP.csv',
'yahoo_/yahoo_sp500/KHC.csv',
'yahoo_/yahoo_sp500/IPG.csv',
'yahoo_/yahoo_sp500/ABNB.csv',
'yahoo_/yahoo_sp500/UNP.csv',
'yahoo_/yahoo_sp500/ALLE.csv',
'yahoo_/yahoo_sp500/ABBV.csv',
'yahoo_/yahoo_sp500/CDAY.csv',
'yahoo_/yahoo_sp500/ORCL.csv',
'yahoo_/yahoo_sp500/ECL.csv',
'yahoo_/yahoo_sp500/ETR.csv',
'yahoo_/yahoo_sp500/EBAY.csv',
'yahoo_/yahoo_sp500/SBUX.csv',
'yahoo_/yahoo_sp500/IR.csv',
'yahoo_/yahoo_sp500/AMT.csv',
'yahoo_/yahoo_sp500/INTU.csv',
'yahoo_/yahoo_sp500/DPZ.csv',
'yahoo_/yahoo_sp500/PAYC.csv',
'yahoo_/yahoo_sp500/CMA.csv',
'yahoo_/yahoo_sp500/PG.csv',
'yahoo_/yahoo_sp500/CAT.csv',
'yahoo_/yahoo_sp500/ODFL.csv',
'yahoo_/yahoo_sp500/MCD.csv',
'yahoo_/yahoo_sp500/MNST.csv',
'yahoo_/yahoo_sp500/AMZN.csv',
'yahoo_/yahoo_sp500/GEHC.csv',
'yahoo_/yahoo_sp500/EG.csv',
'yahoo_/yahoo_sp500/INTC.csv',
'yahoo_/yahoo_sp500/PNR.csv',
'yahoo_/yahoo_sp500/GLW.csv',
'yahoo_/yahoo_sp500/BDX.csv',
'yahoo_/yahoo_sp500/KMI.csv',
'yahoo_/yahoo_sp500/CSGP.csv',
'yahoo_/yahoo_sp500/PWR.csv',
'yahoo_/yahoo_sp500/APTV.csv',
'yahoo_/yahoo_sp500/BBWI.csv',
'yahoo_/yahoo_sp500/DXCM.csv',
'yahoo_/yahoo_sp500/EXR.csv',
'yahoo_/yahoo_sp500/WELL.csv',
'yahoo_/yahoo_sp500/HOLX.csv',
'yahoo_/yahoo_sp500/EXPD.csv',
'yahoo_/yahoo_sp500/GM.csv',
'yahoo_/yahoo_sp500/TXN.csv',
'yahoo_/yahoo_sp500/VRSK.csv',
'yahoo_/yahoo_sp500/FI.csv',
'yahoo_/yahoo_sp500/SJM.csv',
'yahoo_/yahoo_sp500/TMO.csv',
'yahoo_/yahoo_sp500/OXY.csv',
'yahoo_/yahoo_sp500/RL.csv',
'yahoo_/yahoo_sp500/CCI.csv',
'yahoo_/yahoo_sp500/MMM.csv',
'yahoo_/yahoo_sp500/MOS.csv',
'yahoo_/yahoo_sp500/FTNT.csv',
'yahoo_/yahoo_sp500/HSY.csv',
'yahoo_/yahoo_sp500/JNPR.csv',
'yahoo_/yahoo_sp500/DHI.csv',
'yahoo_/yahoo_sp500/ED.csv',
'yahoo_/yahoo_sp500/ES.csv',
'yahoo_/yahoo_sp500/ADSK.csv',
'yahoo_/yahoo_sp500/GL.csv',
'yahoo_/yahoo_sp500/INVH.csv',
'yahoo_/yahoo_sp500/IP.csv',
'yahoo_/yahoo_sp500/EXPE.csv',
'yahoo_/yahoo_sp500/KO.csv',
'yahoo_/yahoo_sp500/PCAR.csv',
'yahoo_/yahoo_sp500/RVTY.csv',
'yahoo_/yahoo_sp500/WDC.csv',
'yahoo_/yahoo_sp500/PYPL.csv',
'yahoo_/yahoo_sp500/NEE.csv',
'yahoo_/yahoo_sp500/UPS.csv',
'yahoo_/yahoo_sp500/ELV.csv',
'yahoo_/yahoo_sp500/EMR.csv',
'yahoo_/yahoo_sp500/MSFT.csv',
'yahoo_/yahoo_sp500/ANSS.csv',
'yahoo_/yahoo_sp500/CTAS.csv',
'yahoo_/yahoo_sp500/BIO.csv',
'yahoo_/yahoo_sp500/UDR.csv',
'yahoo_/yahoo_sp500/CTLT.csv',
'yahoo_/yahoo_sp500/WEC.csv',
'yahoo_/yahoo_sp500/AME.csv',
'yahoo_/yahoo_sp500/IT.csv',
'yahoo_/yahoo_sp500/DD.csv',
'yahoo_/yahoo_sp500/ACN.csv',
'yahoo_/yahoo_sp500/VRSN.csv',
'yahoo_/yahoo_sp500/EW.csv',
'yahoo_/yahoo_sp500/CMG.csv',
'yahoo_/yahoo_sp500/AWK.csv',
'yahoo_/yahoo_sp500/COO.csv',
'yahoo_/yahoo_sp500/SHW.csv',
'yahoo_/yahoo_sp500/HPQ.csv',
'yahoo_/yahoo_sp500/AMAT.csv',
'yahoo_/yahoo_sp500/CCL.csv',
'yahoo_/yahoo_sp500/MLM.csv',
'yahoo_/yahoo_sp500/AVY.csv',
'yahoo_/yahoo_sp500/EVRG.csv',
'yahoo_/yahoo_sp500/EA.csv',
'yahoo_/yahoo_sp500/DE.csv',
'yahoo_/yahoo_sp500/VLTO.csv',
'yahoo_/yahoo_sp500/SPG.csv',
'yahoo_/yahoo_sp500/AMD.csv',
'yahoo_/yahoo_sp500/KLAC.csv',
'yahoo_/yahoo_sp500/NDAQ.csv',
'yahoo_/yahoo_sp500/URI.csv',
'yahoo_/yahoo_sp500/WHR.csv',
'yahoo_/yahoo_sp500/RTX.csv',
'yahoo_/yahoo_sp500/NXPI.csv',
'yahoo_/yahoo_sp500/PNC.csv',
'yahoo_/yahoo_sp500/KMX.csv',
'yahoo_/yahoo_sp500/SEDG.csv',
'yahoo_/yahoo_sp500/WRK.csv',
'yahoo_/yahoo_sp500/MTCH.csv',
'yahoo_/yahoo_sp500/BIIB.csv',
'yahoo_/yahoo_sp500/NVDA.csv',
'yahoo_/yahoo_sp500/CHRW.csv',
'yahoo_/yahoo_sp500/ROP.csv',
'yahoo_/yahoo_sp500/IDXX.csv',
'yahoo_/yahoo_sp500/EXC.csv',
'yahoo_/yahoo_sp500/HES.csv',
'yahoo_/yahoo_sp500/HD.csv',
'yahoo_/yahoo_sp500/ALB.csv',
'yahoo_/yahoo_sp500/VLO.csv',
'yahoo_/yahoo_sp500/AON.csv',
'yahoo_/yahoo_sp500/FDX.csv',
'yahoo_/yahoo_sp500/DG.csv',
'yahoo_/yahoo_sp500/TYL.csv',
'yahoo_/yahoo_sp500/HIG.csv',
'yahoo_/yahoo_sp500/CMS.csv',
'yahoo_/yahoo_sp500/CAG.csv',
'yahoo_/yahoo_sp500/INCY.csv',
'yahoo_/yahoo_sp500/SCHW.csv',
'yahoo_/yahoo_sp500/HSIC.csv',
'yahoo_/yahoo_sp500/AZO.csv',
'yahoo_/yahoo_sp500/AXP.csv',
'yahoo_/yahoo_sp500/HPE.csv',
'yahoo_/yahoo_sp500/DFS.csv',
'yahoo_/yahoo_sp500/SEE.csv',
'yahoo_/yahoo_sp500/HRL.csv',
'yahoo_/yahoo_sp500/SO.csv',
'yahoo_/yahoo_sp500/FRT.csv',
'yahoo_/yahoo_sp500/ZBH.csv',
'yahoo_/yahoo_sp500/CME.csv',
'yahoo_/yahoo_sp500/XOM.csv',
'yahoo_/yahoo_sp500/AMP.csv',
'yahoo_/yahoo_sp500/CVX.csv',
'yahoo_/yahoo_sp500/CMCSA.csv',
'yahoo_/yahoo_sp500/PCG.csv',
'yahoo_/yahoo_sp500/PNW.csv',
'yahoo_/yahoo_sp500/ICE.csv',
'yahoo_/yahoo_sp500/BEN.csv',
'yahoo_/yahoo_sp500/UHS.csv',
'yahoo_/yahoo_sp500/BKR.csv',
'yahoo_/yahoo_sp500/EMN.csv',
'yahoo_/yahoo_sp500/SBAC.csv',
'yahoo_/yahoo_sp500/ROK.csv',
'yahoo_/yahoo_sp500/PTC.csv',
'yahoo_/yahoo_sp500/NRG.csv',
'yahoo_/yahoo_sp500/NSC.csv',
'yahoo_/yahoo_sp500/NKE.csv',
'yahoo_/yahoo_sp500/FIS.csv',
'yahoo_/yahoo_sp500/FANG.csv',
'yahoo_/yahoo_sp500/VTR.csv',
'yahoo_/yahoo_sp500/MAS.csv',
'yahoo_/yahoo_sp500/RF.csv',
'yahoo_/yahoo_sp500/ETSY.csv',
'yahoo_/yahoo_sp500/AMCR.csv',
'yahoo_/yahoo_sp500/TAP.csv',
'yahoo_/yahoo_sp500/MAR.csv',
'yahoo_/yahoo_sp500/XYL.csv',
'yahoo_/yahoo_sp500/CMI.csv',
'yahoo_/yahoo_sp500/MTD.csv',
'yahoo_/yahoo_sp500/BF-B.csv',
'yahoo_/yahoo_sp500/KR.csv',
'yahoo_/yahoo_sp500/PLD.csv',
'yahoo_/yahoo_sp500/IBM.csv',
'yahoo_/yahoo_sp500/USB.csv',
'yahoo_/yahoo_sp500/BSX.csv',
'yahoo_/yahoo_sp500/LKQ.csv',
'yahoo_/yahoo_sp500/LIN.csv',
'yahoo_/yahoo_sp500/ITW.csv',
'yahoo_/yahoo_sp500/EOG.csv',
'yahoo_/yahoo_sp500/KMB.csv',
'yahoo_/yahoo_sp500/PEAK.csv',
'yahoo_/yahoo_sp500/SPGI.csv',
'yahoo_/yahoo_sp500/NEM.csv',
'yahoo_/yahoo_sp500/LULU.csv',
'yahoo_/yahoo_sp500/WFC.csv',
'yahoo_/yahoo_sp500/CTVA.csv',
'yahoo_/yahoo_sp500/EL.csv',
'yahoo_/yahoo_sp500/GS.csv',
'yahoo_/yahoo_sp500/GD.csv',
'yahoo_/yahoo_sp500/CNP.csv',
'yahoo_/yahoo_sp500/PM.csv',
'yahoo_/yahoo_sp500/MCO.csv',
'yahoo_/yahoo_sp500/CLX.csv',
'yahoo_/yahoo_sp500/CAH.csv',
'yahoo_/yahoo_sp500/PANW.csv',
'yahoo_/yahoo_sp500/MPWR.csv',
'yahoo_/yahoo_sp500/DGX.csv',
'yahoo_/yahoo_sp500/AVB.csv',
'yahoo_/yahoo_sp500/DIS.csv',
'yahoo_/yahoo_sp500/CBRE.csv',
'yahoo_/yahoo_sp500/GE.csv',
'yahoo_/yahoo_sp500/HII.csv',
'yahoo_/yahoo_sp500/LDOS.csv',
'yahoo_/yahoo_sp500/ALL.csv',
'yahoo_/yahoo_sp500/ETN.csv',
'yahoo_/yahoo_sp500/ALGN.csv',
'yahoo_/yahoo_sp500/NFLX.csv',
'yahoo_/yahoo_sp500/LEN.csv',
'yahoo_/yahoo_sp500/FITB.csv',
'yahoo_/yahoo_sp500/WST.csv',
'yahoo_/yahoo_sp500/GWW.csv',
'yahoo_/yahoo_sp500/TRGP.csv',
'yahoo_/yahoo_sp500/NTRS.csv',
'yahoo_/yahoo_sp500/FICO.csv',
'yahoo_/yahoo_sp500/AXON.csv',
'yahoo_/yahoo_sp500/CVS.csv',
'yahoo_/yahoo_sp500/AOS.csv',
'yahoo_/yahoo_sp500/FE.csv',
'yahoo_/yahoo_sp500/JPM.csv',
'yahoo_/yahoo_sp500/ABT.csv',
'yahoo_/yahoo_sp500/OMC.csv',
'yahoo_/yahoo_sp500/COF.csv',
'yahoo_/yahoo_sp500/TSCO.csv',
'yahoo_/yahoo_sp500/PH.csv',
'yahoo_/yahoo_sp500/HST.csv',
'yahoo_/yahoo_sp500/JBHT.csv',
'yahoo_/yahoo_sp500/MRNA.csv',
'yahoo_/yahoo_sp500/TSLA.csv',
'yahoo_/yahoo_sp500/MOH.csv',
'yahoo_/yahoo_sp500/ATO.csv',
'yahoo_/yahoo_sp500/COP.csv',
'yahoo_/yahoo_sp500/DHR.csv',
'yahoo_/yahoo_sp500/CNC.csv',
'yahoo_/yahoo_sp500/MCK.csv',
'yahoo_/yahoo_sp500/TXT.csv',
'yahoo_/yahoo_sp500/MTB.csv',
'yahoo_/yahoo_sp500/FDS.csv',
'yahoo_/yahoo_sp500/VTRS.csv',
'yahoo_/yahoo_sp500/AKAM.csv',
'yahoo_/yahoo_sp500/ROL.csv',
'yahoo_/yahoo_sp500/RMD.csv',
'yahoo_/yahoo_sp500/WRB.csv',
'yahoo_/yahoo_sp500/GOOGL.csv',
'yahoo_/yahoo_sp500/BRO.csv',
'yahoo_/yahoo_sp500/ANET.csv',
'yahoo_/yahoo_sp500/PAYX.csv',
'yahoo_/yahoo_sp500/ALK.csv',
'yahoo_/yahoo_sp500/DRI.csv',
'yahoo_/yahoo_sp500/ILMN.csv',
'yahoo_/yahoo_sp500/META.csv',
'yahoo_/yahoo_sp500/AAL.csv',
'yahoo_/yahoo_sp500/COR.csv',
'yahoo_/yahoo_sp500/MAA.csv',
'yahoo_/yahoo_sp500/MMC.csv',
'yahoo_/yahoo_sp500/FOXA.csv',
'yahoo_/yahoo_sp500/POOL.csv',
'yahoo_/yahoo_sp500/CZR.csv',
'yahoo_/yahoo_sp500/FFIV.csv',
'yahoo_/yahoo_sp500/CINF.csv',
'yahoo_/yahoo_sp500/VMC.csv',
'yahoo_/yahoo_sp500/MKTX.csv',
'yahoo_/yahoo_sp500/SRE.csv',
'yahoo_/yahoo_sp500/LHX.csv',
'yahoo_/yahoo_sp500/ORLY.csv',
'yahoo_/yahoo_sp500/IVZ.csv',
'yahoo_/yahoo_sp500/RCL.csv',
'yahoo_/yahoo_sp500/PXD.csv',
'yahoo_/yahoo_sp500/SNPS.csv',
'yahoo_/yahoo_sp500/GOOG.csv',
'yahoo_/yahoo_sp500/EPAM.csv',
'yahoo_/yahoo_sp500/NDSN.csv',
'yahoo_/yahoo_sp500/YUM.csv',
'yahoo_/yahoo_sp500/EQT.csv',
'yahoo_/yahoo_sp500/LYV.csv',
'yahoo_/yahoo_sp500/PFE.csv',
'yahoo_/yahoo_sp500/AVGO.csv',
'yahoo_/yahoo_sp500/DUK.csv',
'yahoo_/yahoo_sp500/REGN.csv',
'yahoo_/yahoo_sp500/CL.csv',
'yahoo_/yahoo_sp500/VFC.csv',
'yahoo_/yahoo_sp500/VZ.csv',
'yahoo_/yahoo_sp500/JCI.csv',
'yahoo_/yahoo_sp500/FSLR.csv',
'yahoo_/yahoo_sp500/AMGN.csv',
'yahoo_/yahoo_sp500/TEL.csv',
'yahoo_/yahoo_sp500/JKHY.csv',
'yahoo_/yahoo_sp500/ADP.csv',
'yahoo_/yahoo_sp500/ON.csv',
'yahoo_/yahoo_sp500/STT.csv',
'yahoo_/yahoo_sp500/RSG.csv',
'yahoo_/yahoo_sp500/IFF.csv',
'yahoo_/yahoo_sp500/CARR.csv',
'yahoo_/yahoo_sp500/TRMB.csv',
'yahoo_/yahoo_sp500/QCOM.csv',
'yahoo_/yahoo_sp500/LYB.csv',
'yahoo_/yahoo_sp500/GIS.csv',
'yahoo_/yahoo_sp500/PHM.csv',
'yahoo_/yahoo_sp500/ROST.csv',
'yahoo_/yahoo_sp500/LUV.csv',
'yahoo_/yahoo_sp500/LW.csv',
'yahoo_/yahoo_sp500/MS.csv',
'yahoo_/yahoo_sp500/CPB.csv',
'yahoo_/yahoo_sp500/OKE.csv',
'yahoo_/yahoo_sp500/BK.csv',
'yahoo_/yahoo_sp500/J.csv',
'yahoo_/yahoo_sp500/SYF.csv',
'yahoo_/yahoo_sp500/CHD.csv',
'yahoo_/yahoo_sp500/HWM.csv',
'yahoo_/yahoo_sp500/MHK.csv',
'yahoo_/yahoo_sp500/TFC.csv',
'yahoo_/yahoo_sp500/DAL.csv',
'yahoo_/yahoo_sp500/APA.csv',
'yahoo_/yahoo_sp500/K.csv',
'yahoo_/yahoo_sp500/AFL.csv',
'yahoo_/yahoo_sp500/CSX.csv',
'yahoo_/yahoo_sp500/NI.csv',
'yahoo_/yahoo_sp500/CPT.csv',
'yahoo_/yahoo_sp500/PFG.csv',
'yahoo_/yahoo_sp500/NCLH.csv',
'yahoo_/yahoo_sp500/ZION.csv',
'yahoo_/yahoo_sp500/RJF.csv',
'yahoo_/yahoo_sp500/HBAN.csv',
'yahoo_/yahoo_sp500/UNH.csv',
'yahoo_/yahoo_sp500/PRU.csv',
'yahoo_/yahoo_sp500/GPC.csv',
'yahoo_/yahoo_sp500/WTW.csv',
'yahoo_/yahoo_sp500/WMB.csv',
'yahoo_/yahoo_sp500/EQR.csv',
'yahoo_/yahoo_sp500/DVA.csv',
'yahoo_/yahoo_sp500/AIG.csv',
'yahoo_/yahoo_sp500/MA.csv',
'yahoo_/yahoo_sp500/HON.csv',
'yahoo_/yahoo_sp500/VICI.csv',
'yahoo_/yahoo_sp500/O.csv',
'yahoo_/yahoo_sp500/NWSA.csv',
'yahoo_/yahoo_sp500/TTWO.csv',
'yahoo_/yahoo_sp500/AES.csv',
'yahoo_/yahoo_sp500/SLB.csv',
'yahoo_/yahoo_sp500/TT.csv',
'yahoo_/yahoo_sp500/TGT.csv',
'yahoo_/yahoo_sp500/AAPL.csv',
'yahoo_/yahoo_sp500/MKC.csv',
'yahoo_/yahoo_sp500/OTIS.csv',
'yahoo_/yahoo_sp500/CEG.csv',
'yahoo_/yahoo_sp500/TDY.csv',
'yahoo_/yahoo_sp500/WY.csv',
'yahoo_/yahoo_sp500/APD.csv',
'yahoo_/yahoo_sp500/GRMN.csv',
'yahoo_/yahoo_sp500/AEE.csv',
'yahoo_/yahoo_sp500/BX.csv',
'yahoo_/yahoo_sp500/HLT.csv',
'yahoo_/yahoo_sp500/DLTR.csv',
'yahoo_/yahoo_sp500/STE.csv',
'yahoo_/yahoo_sp500/HAS.csv',
'yahoo_/yahoo_sp500/TMUS.csv',
'yahoo_/yahoo_sp500/WMT.csv',
'yahoo_/yahoo_sp500/NTAP.csv',
'yahoo_/yahoo_sp500/KIM.csv',
'yahoo_/yahoo_sp500/BAX.csv',
'yahoo_/yahoo_sp500/LMT.csv',
'yahoo_/yahoo_sp500/KEY.csv',
'yahoo_/yahoo_sp500/KEYS.csv',
'yahoo_/yahoo_sp500/BMY.csv',
'yahoo_/yahoo_sp500/PSA.csv',
'yahoo_/yahoo_sp500/WYNN.csv',
'yahoo_/yahoo_sp500/RHI.csv',
'yahoo_/yahoo_sp500/EFX.csv',
'yahoo_/yahoo_sp500/NUE.csv',
'yahoo_/yahoo_sp500/PKG.csv',
'yahoo_/yahoo_sp500/WAB.csv',
'yahoo_/yahoo_sp500/CTSH.csv',
'yahoo_/yahoo_sp500/SWK.csv',
'yahoo_/yahoo_sp500/CRL.csv',
'yahoo_/yahoo_sp500/MU.csv',
'yahoo_/yahoo_sp500/TRV.csv',
'yahoo_/yahoo_sp500/BRK-B.csv',
'yahoo_/yahoo_sp500/L.csv',
'yahoo_/yahoo_sp500/AEP.csv',
'yahoo_/yahoo_sp500/CI.csv',
'yahoo_/yahoo_sp500/DOW.csv',
'yahoo_/yahoo_sp500/CDW.csv',
'yahoo_/yahoo_sp500/BALL.csv',
'yahoo_/yahoo_sp500/JNJ.csv',
'yahoo_/yahoo_sp500/WM.csv',
'yahoo_/yahoo_sp500/DOV.csv',
'yahoo_/yahoo_sp500/CRM.csv',
'yahoo_/yahoo_sp500/PGR.csv',
'yahoo_/yahoo_sp500/WAT.csv',
'yahoo_/yahoo_sp500/IEX.csv',
'yahoo_/yahoo_sp500/BWA.csv',
'yahoo_/yahoo_sp500/LRCX.csv',
'yahoo_/yahoo_sp500/BLK.csv',
'yahoo_/yahoo_sp500/PPL.csv'
]


# Assuming 'file_paths' is a list of file paths to process
# Since I don't have access to the actual CSV files, I'll just outline how the loop would work





processed_files = []

for file_path in file_paths:
    try:
        preprocessed_file_path = preprocess_data(file_path)
        processed_files.append(preprocessed_file_path)
        print(f"Preprocessed file saved to: {preprocessed_file_path}")
    except Exception as e:
        print(f"An error occurred while processing {file_path}: {e}")

# This will print out the paths of the preprocessed files or any errors that occur
processed_files


    
    
# Example usage:
#preprocessed_file_path, dividend_file_path = preprocess_data(file_path)
#print(f"Preprocessed file saved to: {preprocessed_file_path}")
#print(f"Dividend file saved to: {dividend_file_path}")


Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/CSCO_preprocessed.csv
Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/UAL_preprocessed.csv
Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/TROW_preprocessed.csv
Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/ISRG_preprocessed.csv
Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/NVR_preprocessed.csv
Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/TPR_preprocessed.csv
Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/DVN_preprocessed.csv
Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/CE_preprocessed.csv
Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/MRO_preprocessed.csv
Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/BA_preprocessed.csv
Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/VRTX_preprocessed.csv
Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/GILD_preprocessed.csv
Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/EQIX_preprocessed

Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/NOC_preprocessed.csv
Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/BXP_preprocessed.csv
Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/KHC_preprocessed.csv
Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/IPG_preprocessed.csv
Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/ABNB_preprocessed.csv
Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/UNP_preprocessed.csv
Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/ALLE_preprocessed.csv
Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/ABBV_preprocessed.csv
Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/CDAY_preprocessed.csv
Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/ORCL_preprocessed.csv
Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/ECL_preprocessed.csv
Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/ETR_preprocessed.csv
Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/EBAY_preprocess

Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/PNC_preprocessed.csv
Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/KMX_preprocessed.csv
Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/SEDG_preprocessed.csv
Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/WRK_preprocessed.csv
Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/MTCH_preprocessed.csv
Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/BIIB_preprocessed.csv
Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/NVDA_preprocessed.csv
Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/CHRW_preprocessed.csv
Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/ROP_preprocessed.csv
Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/IDXX_preprocessed.csv
Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/EXC_preprocessed.csv
Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/HES_preprocessed.csv
Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/HD_preprocesse

Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/NTRS_preprocessed.csv
Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/FICO_preprocessed.csv
Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/AXON_preprocessed.csv
Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/CVS_preprocessed.csv
Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/AOS_preprocessed.csv
Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/FE_preprocessed.csv
Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/JPM_preprocessed.csv
Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/ABT_preprocessed.csv
Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/OMC_preprocessed.csv
Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/COF_preprocessed.csv
Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/TSCO_preprocessed.csv
Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/PH_preprocessed.csv
Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/HST_preprocessed.c

Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/ZION_preprocessed.csv
Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/RJF_preprocessed.csv
Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/HBAN_preprocessed.csv
Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/UNH_preprocessed.csv
Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/PRU_preprocessed.csv
Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/GPC_preprocessed.csv
Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/WTW_preprocessed.csv
Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/WMB_preprocessed.csv
Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/EQR_preprocessed.csv
Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/DVA_preprocessed.csv
Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/AIG_preprocessed.csv
Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/MA_preprocessed.csv
Preprocessed file saved to: ./yahoo_/yahoo_preprocessed/HON_preprocessed.cs

['./yahoo_/yahoo_preprocessed/CSCO_preprocessed.csv',
 './yahoo_/yahoo_preprocessed/UAL_preprocessed.csv',
 './yahoo_/yahoo_preprocessed/TROW_preprocessed.csv',
 './yahoo_/yahoo_preprocessed/ISRG_preprocessed.csv',
 './yahoo_/yahoo_preprocessed/NVR_preprocessed.csv',
 './yahoo_/yahoo_preprocessed/TPR_preprocessed.csv',
 './yahoo_/yahoo_preprocessed/DVN_preprocessed.csv',
 './yahoo_/yahoo_preprocessed/CE_preprocessed.csv',
 './yahoo_/yahoo_preprocessed/MRO_preprocessed.csv',
 './yahoo_/yahoo_preprocessed/BA_preprocessed.csv',
 './yahoo_/yahoo_preprocessed/VRTX_preprocessed.csv',
 './yahoo_/yahoo_preprocessed/GILD_preprocessed.csv',
 './yahoo_/yahoo_preprocessed/EQIX_preprocessed.csv',
 './yahoo_/yahoo_preprocessed/TER_preprocessed.csv',
 './yahoo_/yahoo_preprocessed/PODD_preprocessed.csv',
 './yahoo_/yahoo_preprocessed/MDT_preprocessed.csv',
 './yahoo_/yahoo_preprocessed/V_preprocessed.csv',
 './yahoo_/yahoo_preprocessed/QRVO_preprocessed.csv',
 './yahoo_/yahoo_preprocessed/A_preprocess

In [85]:
#import os
#import shutil
#
## Set your base directory here
#base_directory = './yahoo_appl/preprocessed_data/'  # Replace with the path to your target directory
#
## Define the directories
#processed_subfolder = os.path.join(base_directory, 'processed_files')
#dividends_subfolder = os.path.join(base_directory, 'dividends_files')
#
## Create the subfolders if they don't exist
#os.makedirs(processed_subfolder, exist_ok=True)
#os.makedirs(dividends_subfolder, exist_ok=True)
#
## Iterate over each file in the base directory
#for filename in os.listdir(base_directory):
#    # Check if the file is a .csv file
#    if filename.endswith('.csv'):
#        # Move processed.csv files to the 'processed_files' subfolder
#        if 'preprocessed.csv' in filename:
#            shutil.move(os.path.join(base_directory, filename), processed_subfolder)
#        
#        # Move dividends.csv files to the 'dividends_files' subfolder
#        elif 'dividends.csv' in filename:
#            shutil.move(os.path.join(base_directory, filename), dividends_subfolder)

In [73]:
import pandas as pd
import os
import json
from pathlib import Path

# Function to analyze a single CSV file
def analyze_csv(file_path):
    try:
        df = pd.read_csv(file_path)
        analyze_dataframe(df, file_path)
    except Exception as e:
        print(f'Could not read {file_path}: {e}')

# Function to analyze a single JSON file with nested structure details
def analyze_json(file_path):
    try:
        with open(file_path, 'r') as f:
            data = json.load(f)
            df = pd.json_normalize(data, sep='_')
            analyze_dataframe(df, file_path)
            if isinstance(data, dict):
                print_nested_json_structure(data, level=0)
            elif isinstance(data, list) and data:
                print_nested_json_structure(data[0], level=0)  # Assumes first element is representative of the list
    except Exception as e:
        print(f'Could not read {file_path}: {e}')

# Helper function to print nested JSON structure
def print_nested_json_structure(data, level):
    indent = '  ' * level
    if isinstance(data, dict):
        for key, value in data.items():
            if isinstance(value, dict) or isinstance(value, list):
                print(f'{indent}{key}:')
                print_nested_json_structure(value, level + 1)
            else:
                print(f'{indent}{key}: {type(value).__name__}')
    elif isinstance(data, list):
        print(f'{indent}List of {len(data)} items: {type(data[0]).__name__}')
        if data and isinstance(data[0], (dict, list)):
            print_nested_json_structure(data[0], level + 1)

# General function to analyze DataFrames
def analyze_dataframe(df, file_path):
    column_details = df.dtypes.apply(lambda x: x.name).to_dict()
    num_rows, num_columns = df.shape
    nan_values = df.isna().sum().to_dict()

    print(file_path)
    #print(f'File: {file_path}')
    #print(f'Columns and Types: {column_details}')
    #print(f'Number of Rows: {num_rows}')
    #print(f'Number of Columns: {num_columns}')
    #print(f'NaN values per column: {nan_values}')
    #print('-----------------------------------\n')

# Function to traverse directories and find CSV and JSON files
def find_files(directory):
    pathlist = Path(directory).rglob('*.*')
    for path in pathlist:
        # Analyze files based on extension
        if path.suffix.lower() == '.csv':
            analyze_csv(str(path))
        elif path.suffix.lower() == '.json':
            analyze_json(str(path))

# Replace 'your_directory_path' with the path to the directory containing your CSV and JSON files
your_directory_path = './yahoo_/yahoo_sp500/'
find_files(your_directory_path)


yahoo_/yahoo_sp500/CSCO.csv
yahoo_/yahoo_sp500/UAL.csv
yahoo_/yahoo_sp500/TROW.csv
yahoo_/yahoo_sp500/ISRG.csv
yahoo_/yahoo_sp500/NVR.csv
yahoo_/yahoo_sp500/TPR.csv
yahoo_/yahoo_sp500/DVN.csv
yahoo_/yahoo_sp500/CE.csv
yahoo_/yahoo_sp500/MRO.csv
yahoo_/yahoo_sp500/BA.csv
yahoo_/yahoo_sp500/VRTX.csv
yahoo_/yahoo_sp500/GILD.csv
yahoo_/yahoo_sp500/EQIX.csv
yahoo_/yahoo_sp500/TER.csv
yahoo_/yahoo_sp500/PODD.csv
yahoo_/yahoo_sp500/MDT.csv
yahoo_/yahoo_sp500/V.csv
yahoo_/yahoo_sp500/QRVO.csv
yahoo_/yahoo_sp500/A.csv
yahoo_/yahoo_sp500/FOX.csv
yahoo_/yahoo_sp500/FLT.csv
yahoo_/yahoo_sp500/MO.csv
yahoo_/yahoo_sp500/CTRA.csv
yahoo_/yahoo_sp500/SWKS.csv
yahoo_/yahoo_sp500/ENPH.csv
yahoo_/yahoo_sp500/MCHP.csv
yahoo_/yahoo_sp500/CDNS.csv
yahoo_/yahoo_sp500/MSCI.csv
yahoo_/yahoo_sp500/CHTR.csv
yahoo_/yahoo_sp500/EIX.csv
yahoo_/yahoo_sp500/KDP.csv
yahoo_/yahoo_sp500/BBY.csv
yahoo_/yahoo_sp500/GEN.csv
yahoo_/yahoo_sp500/WBA.csv
yahoo_/yahoo_sp500/LVS.csv
yahoo_/yahoo_sp500/HCA.csv
yahoo_/yahoo_sp500/A

yahoo_/yahoo_sp500/ETN.csv
yahoo_/yahoo_sp500/ALGN.csv
yahoo_/yahoo_sp500/NFLX.csv
yahoo_/yahoo_sp500/LEN.csv
yahoo_/yahoo_sp500/FITB.csv
yahoo_/yahoo_sp500/WST.csv
yahoo_/yahoo_sp500/GWW.csv
yahoo_/yahoo_sp500/TRGP.csv
yahoo_/yahoo_sp500/NTRS.csv
yahoo_/yahoo_sp500/FICO.csv
yahoo_/yahoo_sp500/AXON.csv
yahoo_/yahoo_sp500/CVS.csv
yahoo_/yahoo_sp500/AOS.csv
yahoo_/yahoo_sp500/FE.csv
yahoo_/yahoo_sp500/JPM.csv
yahoo_/yahoo_sp500/ABT.csv
yahoo_/yahoo_sp500/OMC.csv
yahoo_/yahoo_sp500/COF.csv
yahoo_/yahoo_sp500/TSCO.csv
yahoo_/yahoo_sp500/PH.csv
yahoo_/yahoo_sp500/HST.csv
yahoo_/yahoo_sp500/JBHT.csv
yahoo_/yahoo_sp500/MRNA.csv
yahoo_/yahoo_sp500/TSLA.csv
yahoo_/yahoo_sp500/MOH.csv
yahoo_/yahoo_sp500/ATO.csv
yahoo_/yahoo_sp500/COP.csv
yahoo_/yahoo_sp500/DHR.csv
yahoo_/yahoo_sp500/CNC.csv
yahoo_/yahoo_sp500/MCK.csv
yahoo_/yahoo_sp500/TXT.csv
yahoo_/yahoo_sp500/MTB.csv
yahoo_/yahoo_sp500/FDS.csv
yahoo_/yahoo_sp500/VTRS.csv
yahoo_/yahoo_sp500/AKAM.csv
yahoo_/yahoo_sp500/ROL.csv
yahoo_/yahoo_sp50

In [89]:
import pandas as pd



# Specify your desired file path
file_path = './yahoo_/yahoo_preprocessed/AAL_preprocessed.csv'

# Save the DataFrame to a CSV file at the specified path
data.to_csv(file_path, index=False)  # Set index=False if you do not want to save the index


In [90]:
data.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'MA_5',
       'MA_10', 'MA_20', 'MA_50', 'MA_100', 'Pct_change', 'Volatility',
       'Day_of_week', 'RSI', 'MACD', 'MACD_signal', 'Bollinger_mavg',
       'Bollinger_hband', 'Bollinger_lband', 'Lag_1', 'Lag_2', 'Lag_3',
       'Volume_MA_10', 'Volume_pct_MA', 'High_Low_Range',
       'Price_Change_Direction'],
      dtype='object')